# <center> Scallion Pancake Power and Current Budget </center>
### <center> By Noah Stiegler and Daniel Opara </center>
#### <center> Created Feburary 8, 2024 </center>
#### <center> Updated March 24, 2024 </center>

The notebook contains the calculations done to find the power budget and overall current draw of the PCB throughout the entirety of its flight on CARM.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

In [ ]:
import seaborn as sns

sns.set_theme(context='notebook', style='darkgrid', palette='colorblind',
              font='sans-serif', font_scale=1, color_codes=True, rc=None)

## Introduction: Why Bother?

Calculating power and current draw is a crucial aspect of designing electronic systems as it serves many purposes to ensure optimal performance, safety, and financial expenditure. 

It aids in figuring out the efficiency of a circuit or device and is especially helpful for battery-operated devices or energy-sensitive applications. By understanding how much power a circuit consumes, we can select appropriate components such as power supplies, batteries, and voltage regulators, ensuring they can handle the load and operate reliably. This knowledge also helps in budgeting and planning for energy usage, which is essential for projects with power constraints or limited power sources. Furthermore, accurate calculations prevent overloading components, which can lead to overheating, failure, or safety hazards. In addition to all of this, compliance with power consumption or efficiency standards may be necessary for certain applications—in our case, there are various contraints we have on our PCB for Spaceport Amarica—making it imperative to calculate power and current draw to ensure adherence to these standards.

## System Overview

The PCB consists of six main breakout boards/sub-components:
- [Adafruit Feather M0 LoRa](https://learn.adafruit.com/adafruit-feather-m0-radio-with-lora-radio-module)
- [Adafruit Ultimate GPS module](https://learn.adafruit.com/adafruit-ultimate-gps/overview)
- [Micro SD Card Breakout Board](https://learn.adafruit.com/adafruit-micro-sd-breakout-board-card-tutorial/introduction)
- [Adafruit LSM9DS1 (IMU)](https://learn.adafruit.com/adafruit-lsm9ds1-accelerometer-plus-gyro-plus-magnetometer-9-dof-breakout/)
- [Adafruit BMP388 (Barometer)](https://learn.adafruit.com/adafruit-bmp388-bmp390-bmp3xx)
- [Adafruit MCP9808 (Temp. sensor)](https://learn.adafruit.com/adafruit-mcp9808-precision-i2c-temperature-sensor-guide/overview)

There are various other components that play into the power and current draw calculations, however, their impact is minimal so they are not used in our models.

For current draw

In [60]:
# The board consists of six sub-boards, the GPS, SD card reader, Feather 
# transceier, IMU, Barometer, and Temp sensor
# Here we build functions to model their power draw
# I'll model the power draw as the current each daughter board is drawing
# times the amount of time it's drawing that current, in mAh (milliamphours)
# Because each board uses 3.3V, the energy it uses is just the current-time 
# multiplied by 3.3V. However, using amp hours is more useful for modeling 
# battery life, which is why we're using it here
# I use astropy.units to make sure all the unit calculations are correct
# and return as the correct units
mAh = u.milliamp * u.hour

In [ ]:
# Small helper function to check the input is of unit time
def assert_time_unit(value):
        try:
                value.to(u.s)
        except:
                raise Exception("Parameter not a time unit")

In [67]:

# GPS (SPI)
#       Polled at 10hz
#       3.3V @ 25mA (acquisition) & 20mA (tracking) - assume tracking (acquisition higher power is marginal)
#       Source: https://cdn-shop.adafruit.com/product-files/746/CD+PA1616S+Datasheet.v03.pdf
def GPS_energy(time, acquiringSat=False):
        assert_time_unit(time)
        current = 25 * u.milliamp if acquiringSat else 20 * u.milliamp
        return (current * time).to(mAh)

# For example, if we were going to acquire satellites for 10 seconds
# then track for an hour, we would use
energy = (GPS_energy(10 * u.s, acquiringSat=True) + GPS_energy(1 * u.hr)) * (3.3 * u.volt)
energy = energy.to(u.Joule)
print(energy)


238.42499999999998 J


In [69]:

# SD Card Reader (SPI)
#       3.3V @ 100mA (very rough average)
#       Definitely 3.3V, but the card read/write will use way more current
#       (up to 150mA) when actively writing data continuously, and probably
#       a lot less if it's not, but that data isn't super available, so I'll
#       just take 100mA as a baseline, which is probably a huge overestimate
#       Source: https://cdn-learn.adafruit.com/downloads/pdf/adafruit-micro-sd-breakout-board-card-tutorial.pdf & an SD card page
def SD_energy(time):
        assert_time_unit(time)
        return ((100 * u.milliamp) * time).to(mAh)

# For example, if we were going to write data for an hour we would use
energy = SD_energy(1 * u.hr) * (3.3 * u.volt)
energy = energy.to(u.Joule)
print(energy)


1188.0 J


In [71]:

# Feather Transceiver (SPI)
#       3.3V @ 13mA when listening, 11mA when sleeping, 
#       and call the current draw 90mA at 13dBm + PA_BOOST (high radio power
#       to get long range signal)
#       source: https://cdn-shop.adafruit.com/product-files/3179/sx1276_77_78_79.pdf table 10 pg 19
def transceiver_energy(time, mode="listen"):
        # Check input
        assert_time_unit(time)
        acceptable_modes = ["listen", "sleep", "transmit"]
        assert(mode in acceptable_modes)

        # Find the right current
        if mode == acceptable_modes[0]: # Listen
                current = 13 * u.milliamp
        elif mode == acceptable_modes[1]: # Sleep
                current = 11 * u.milliamp
        elif mode == acceptable_modes[2]: # Transmit
                current = 90 * u.milliamp
        else:
                raise Exception("Mode Error")
        
        return (current * time).to(mAh)

# Here I'll create a function which figures out how long the transceiver
# will be transmitting to send a given amount of data
def transmission_time(bits):
        bitrate = 19200 * u.Hz # bits per second
        return (bits / bitrate).to(u.s)

# And here we'll see how much energy it takes to transmit a certain
# amount of data in a certain amount of time, sleeping the radio 
# inbetween
def energy_to_transmit(bits, time):
        assert_time_unit(time)

        # Figure out how long we need the transmitter on
        time_transmitting = transmission_time(bits)

        # Make sure we have enough time to transmit at our bitrate
        assert(time_transmitting <= time)

        # Figure out how much energy we need to transmit for that long
        energy_on = transceiver_energy(time_transmitting, mode="transmit")

        # Figure out how much energy we use while sleeping and not transmitting
        energy_sleep = transceiver_energy(time - time_transmitting, mode="sleep")

        return (energy_on + energy_sleep)


# For example, if we were going to transmit 252 bits (one packet) per second
# and sleep for the rest of the time for an hour on the launchpad that
# would use
bits_to_transmit = 252 * 60 * 60 # 252 bits per second, 60s/min, 60min/hr
energy = energy_to_transmit(bits_to_transmit, 1 * u.hr) * (3.3 * u.volt)
energy = energy.to(u.joule)
print(energy)

142.998075 J


In [72]:

# IMU (I2C)
#       3.3V @ 4.3mA
#       Source: https://www.st.com/resource/en/datasheet/lsm9ds1.pdf table 10 pg 20
def IMU_energy(time):
        assert_time_unit(time)
        return ((4.3 * u.milliamp) * time).to(mAh)

# For example, if we were going to power the IMU for an hour it would take
energy = IMU_energy(1 * u.hr) * (3.3 * u.Volt)
energy = energy.to(u.Joule)
print(energy)

51.083999999999996 J


In [73]:

# Barometer (I2C)
#       Polled at 50hz
#       Takes 20uA @ 1Hz 
#       source: https://www.bosch-sensortec.com/media/boschsensortec/downloads/datasheets/bst-bmp388-ds001.pdf table 21 pg 26
#       3.3V @ 1mA
def barometer_energy(time):
        assert_time_unit(time)
        poll_rate = 50 * u.Hz
        return ((20 * u.microamp / u.Hz) * poll_rate * time).to(mAh)

# For example, if we were going to have the barometer polling for an hour
# it would take
energy = barometer_energy(1 * u.hr) * (3.3 * u.Volt)
energy = energy.to(u.Joule)
print(energy)

11.879999999999999 J


In [74]:

# Temperature Sensor (I2C) & its amplifier
#       3.3V @ 200uA
#       Source: https://cdn-learn.adafruit.com/downloads/pdf/adafruit-mcp9808-precision-i2c-temperature-sensor-guide.pdf
def temp_energy(time):
        assert_time_unit(time)
        return ((200 * u.microamp) * time).to(mAh)

# For example, if we were going to run the temperature sensor for an hour it
# would take
energy = temp_energy(1 * u.hr) * (3.3 * u.Volt)
energy = energy.to(u.Joule)
print(energy)

2.3760000000000003 J


In [59]:
# Let's simulate three phases: launchpad, flight, and recovery
# I'll get the current-time draw at each phase
# The energy can be calculated by multiplying by 3.3V

# On the launchpad, the sensors are powered on and logging data, but only
# transmitting every second. We also need to do any setup. We'll be on the
# launchpad for an hour
energy_from_setup = (GPS_energy(10 * u.s, acquiringSat=True) + # GPS finds satellites
                     GPS_energy(1 * u.hr - 10 * u.s) + # Gps tracks
                     SD_energy(1 * u.hr) + # SD card writes - SUPER rough approximation
                     ).to(kWh)
### TODO HERE -- NOT DONE

# During flight, the sensors are powered on and transmitting at 30 hz
# In recovery, the sensors are powered on and transmitting at 1 hz again
